<h1>Final Project Tutorial - Taeyoon Kim, Victor Chang</h1>

<p style="font-size:14px">For this tutorial, we are going looking at the <a href="https://www.kaggle.com/c/quora-question-pairs/leaderboard">Quora Questions Pair</a> competition on Kaggle. Quora is a website where people can ask questions and they attract over 100 million users each month. As a result, there are many simliar questions on Quora that don't need to be answered again. We are going to go down the data science pipe line and build a Log-Regression Prediction model.</p>



In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import preprocessing

In [25]:
shrooms_df = pd.read_csv('mushrooms.csv')

train_df = shrooms_df.head(1000)
test_df = shrooms_df.tail(1000)

train_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [45]:
le = preprocessing.LabelEncoder()
le.fit(np.unique(train_df.values))
list(le.classes_)
train_df = train_df.apply(le.transform)

y_train = train_df['class']
x_train = train_df[list(train_df.columns[1:])]

decesion_tree = DecisionTreeClassifier(min_samples_split=20, random_state=99)
decesion_tree.fit(x_train, y_train)

with open("dt.dot", 'w') as f:
    export_graphviz(decesion_tree, out_file=f, feature_names=train_df.columns[1:])
    command = ['dot', '-Tpng', 'dt.dot', '-o', 'dt.png']
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")
        
